In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import pandas as pd
from prunable_modules import MaskedLinearLayer, MaskedConvLayer, ResBlock

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

cuda:0


In [2]:
def load_imagenette(BS):
    train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation(20),
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

    val_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


    train_path='/home/florian/data/imagenette2/train'
    imagenette_train = torchvision.datasets.ImageFolder(
        root=train_path,
        transform=train_transforms
    )
    val_path='/home/florian/data/imagenette2/val'
    imagenette_val = torchvision.datasets.ImageFolder(
        root=val_path,
        transform=val_transforms
    )

    train_loader = torch.utils.data.DataLoader(imagenette_train,
                                              batch_size=BS,
                                              shuffle=True)
    val_loader = torch.utils.data.DataLoader(imagenette_val,
                                              batch_size=BS,
                                              shuffle=True)
    return train_loader, val_loader

In [3]:
train_loader, val_loader = load_imagenette(4)

In [4]:
class ImageNetResNet(nn.Module):
    def __init__(self):
        super(ImageNetResNet, self).__init__()
        self.c1 = MaskedConvLayer((64, 3, 7, 7), padding=(3,3),bias=False, stride=2)
        self.p1 = nn.MaxPool2d((3,3), stride=(2,2), padding=(1))
        self.r1 = ResBlock(64,64,padding=1)
        self.r2 = ResBlock(64,64,padding=1)
        self.r3 = ResBlock(64,128,stride=2,padding=1)
        self.r4 = ResBlock(128,128,padding=1)
        self.r5 = ResBlock(128,256,stride=2,padding=1)
        self.r6 = ResBlock(256,256,padding=1)
        self.r7 = ResBlock(256,512,stride=2,padding=1)
        self.r8 = ResBlock(512,512,padding=1)
        self.p2 = nn.AvgPool2d(7)
        self.d1 = MaskedLinearLayer((512,10), activation=None)
        self.conv_weights, self.conv_masks, self.fully_connected_weights, self.fully_connected_masks = self.identify_layers()
        self.train_stats = pd.DataFrame(columns=('epoch', 'train_loss', 'train_accuracy', 'validation_loss', 'validation_accuracy', 'duration', 'criterion', 'optimizer', 'method', 'learning_rate', 'batchsize'))
    
    def forward(self, inputs):
        x = self.c1(inputs)
        x = self.p1(x)
        x = self.r1(x)
        x = self.r2(x)
        x = self.r3(x)
        x = self.r4(x)
        x = self.r5(x)
        x = self.r6(x)
        x = self.r7(x)
        x = self.r8(x)
        x = self.p2(x)
        x = x.view(x.shape[0], x.shape[1])
        x = self.d1(x)
        return (x)
    
    def identify_layers(self):
        conv_weights, conv_masks, fully_connected_weights, fully_connected_masks = [], [], [], []
        for key in self.state_dict().keys():
            if key[:1] == 'c' and 'weights' in key or key[:1] == 'r' and key[3] == 'c' and 'weights' in key:
                conv_weights.append(key)
            if key[:1] == 'c' and 'mask' in key or key[:1] == 'r' and key[3] == 'c' and 'mask' in key:
                conv_masks.append(key)
            if key[:2] == 'fc' and 'weights' in key:
                fully_connected_weights.append(key)
            if key[:2] == 'fc' and 'mask' in key:
                fully_connected_masks.append(key)
        print(conv_masks)
        return conv_weights, conv_masks, fully_connected_weights, fully_connected_masks


    def prune_magnitude_local_unstruct(self, ratio, device, scope='layer'):
        def prune_conv_layers_layerwise(self, ratio, params, device):
            for i, _ in enumerate(self.conv_weights):
                weight_layer = params[self.conv_weights[i]]
                mask_layer = params[self.conv_masks[i]]
                shape = weight_layer.shape
                flat_weights = weight_layer.view(-1)
                flat_masks = mask_layer.view(-1)
                no_of_weights_to_prune = int(round(ratio * len(flat_weights)))
                indices_to_delete = torch.argsort(torch.abs(flat_weights))[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    flat_masks[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                params[self.conv_weights[i]] = flat_weights.view(shape)
                params[self.conv_masks[i]] = flat_masks.view(shape)
            return params

        def prune_conv_layers_filterwise(self, ratio, params, device):
            for i, _ in enumerate(self.conv_weights):
                weight_layer = params[self.conv_weights[i]]
                mask_layer = params[self.conv_masks[i]]
                for j,_ in enumerate(weight_layer):
                    filt = weight_layer[j]
                    filter_mask = mask_layer[j]
                    shape = weight_layer[j].shape
                    flat_weights = filt.view(-1)
                    flat_mask = filter_mask.view(-1)
                    no_of_weights_to_prune = int(round(ratio * len(filt)))
                    indices_to_delete = torch.argsort(torch.abs(filt))[:no_of_weights_to_prune]
                    for idx_to_delete in indices_to_delete:
                        flat_mask[idx_to_delete] = 0
                        flat_weights[idx_to_delete] = 0

                    weight_layer[j] = flat_weights.view(shape)
                    mask_layer[j] = flat_mask.view(shape)

                params[self.conv_weights[i]] = weight_layer.view(shape)
                params[self.conv_masks[i]] = mask_layer.view(shape)
            return params

        def prune_dense_layers(self, ratio, params, device):
            for i, _ in enumerate(self.fully_connected_weights):
                weight_layer = params[self.fully_connected_weights[i]]
                mask_layer = params[self.fully_connected_masks[i]]
                flat_weights = weight_layer.view(-1)
                flat_mask = mask_layer.view(-1)
                shape = weight_layer.shape

                no_of_weights_to_prune = int(round(len(flat_weights)*ratio))
                indices_to_delete = torch.argsort(torch.abs(flat_weights))[:no_of_weights_to_prune]
                for idx_to_delete in indices_to_delete:
                    flat_mask[idx_to_delete] = 0
                    flat_weights[idx_to_delete] = 0
                params[self.fully_connected_weights[i]] = flat_weights.view(shape)
                params[self.fully_connected_masks[i]] = flat_mask.view(shape)
            return params

        params = self.state_dict()
        if scope == 'layer':
            params = prune_conv_layers_layerwise(self,ratio, params, device)
        if scope == 'filter':
            params = prune_conv_layers_filterwise(self,ratio, params, device)
        if scope != 'filter' and scope != 'layer':
            raise Exception('scope should be one of "layer" and "filter"')
        params = prune_dense_layers(self,ratio, params, device)
        self.load_state_dict(params)
        return params

In [5]:
model = ImageNetResNet().to(device)

['c1.mask', 'r1.c1.mask', 'r1.c2.mask', 'r2.c1.mask', 'r2.c2.mask', 'r3.c1.mask', 'r3.c2.mask', 'r3.c3.mask', 'r4.c1.mask', 'r4.c2.mask', 'r5.c1.mask', 'r5.c2.mask', 'r5.c3.mask', 'r6.c1.mask', 'r6.c2.mask', 'r7.c1.mask', 'r7.c2.mask', 'r7.c3.mask', 'r8.c1.mask', 'r8.c2.mask']


In [6]:
model.conv_masks

['c1.mask',
 'r1.c1.mask',
 'r1.c2.mask',
 'r2.c1.mask',
 'r2.c2.mask',
 'r3.c1.mask',
 'r3.c2.mask',
 'r3.c3.mask',
 'r4.c1.mask',
 'r4.c2.mask',
 'r5.c1.mask',
 'r5.c2.mask',
 'r5.c3.mask',
 'r6.c1.mask',
 'r6.c2.mask',
 'r7.c1.mask',
 'r7.c2.mask',
 'r7.c3.mask',
 'r8.c1.mask',
 'r8.c2.mask']

In [7]:
def _evaluate_model(model, data_loader, device, criterion):
    correct = 0
    total = 0
    acc_loss = 0.0
    avg_loss = 0.0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            #print(i)
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if criterion != None:
                loss = criterion(outputs, labels)
                acc_loss += loss.item() 
                avg_loss = acc_loss / (i+1)
            #print(outputs)
            _, predicted = torch.max(outputs.data, 1)
            #print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    model.train()
    return accuracy, avg_loss

def fit(model, train_loader, val_loader, epochs, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    total_time = 0
    epochs_trained = 0
    train_loss_hist, train_acc_hist, val_loss_hist, val_acc_hist = [], [], [], []
    for epoch in range(epochs):  # loop over the dataset multiple times
        t0 = time.time()
        acc_epoch_loss, avg_epoch_loss, epoch_accuracy, acc_epoch_accuracy = 0.0, 0.0, 0.0, 0.0
        
        
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            batchsize = labels.size(0)
            correct = (predicted == labels).sum().item()
            accuracy = 100 * correct / batchsize
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

                # print statistics
            acc_epoch_loss += loss.item() 
            avg_epoch_loss = acc_epoch_loss / (i+1)
            acc_epoch_accuracy += accuracy
            avg_epoch_accuracy = acc_epoch_accuracy / (i+1)
            if i%10 == 0:
                print('[%d, %5d] loss: %.5f, train_accuracy: %.2f' %(epoch + 1, i + 1, loss.item(), accuracy))
        t1 = time.time()
        total_time += t1 - t0
        accuracy, loss = _evaluate_model(model, val_loader, device, criterion)
        #print('duration:', t1-t0,'- train loss: ',avg_epoch_loss,' - train accuracy: ',avg_epoch_accuracy,' - validation accuracy: ', accuracy,' - validation loss: ', loss)
        print('duration: %d s - train loss: %.5f - train accuracy: %.2f - validation loss: %.2f - validation accuracy: %.2f ' %(t1-t0, avg_epoch_loss, avg_epoch_accuracy, loss, accuracy))
        train_loss_hist.append(avg_epoch_loss)
        train_acc_hist.append(avg_epoch_accuracy)
        val_loss_hist.append(loss)
        val_acc_hist.append(accuracy)
        data = {
            'epoch': epoch+1,
            'train_loss':avg_epoch_loss, 
            'train_accuracy':avg_epoch_accuracy,
            'validation_loss':loss,
            'validation_accuracy':accuracy,
            'duration':total_time,
            'criterion':criterion,
            'optimizer':optimizer,
            'method': 'standard',
            'batchsize': len(next(iter(train_loader))[1])
        }
        
    model.train_stats = model.train_stats.append(data, ignore_index=True)
    print('Finished Training')
    
    return model.train_stats

In [ ]:
fit(model, train_loader, val_loader, 100, device)

[1,     1] loss: 4.12376, train_accuracy: 0.00
[1,    11] loss: 15.62767, train_accuracy: 25.00
[1,    21] loss: 39.67447, train_accuracy: 0.00
[1,    31] loss: 4.23959, train_accuracy: 50.00
[1,    41] loss: 17.00320, train_accuracy: 25.00
[1,    51] loss: 8.30997, train_accuracy: 25.00
[1,    61] loss: 5.45977, train_accuracy: 50.00
[1,    71] loss: 9.59836, train_accuracy: 0.00
[1,    81] loss: 7.96904, train_accuracy: 0.00
[1,    91] loss: 18.74900, train_accuracy: 0.00
[1,   101] loss: 3.24320, train_accuracy: 0.00
[1,   111] loss: 7.06145, train_accuracy: 50.00
[1,   121] loss: 15.36932, train_accuracy: 0.00
[1,   131] loss: 4.37726, train_accuracy: 0.00
[1,   141] loss: 5.24490, train_accuracy: 0.00
[1,   151] loss: 4.12448, train_accuracy: 25.00
[1,   161] loss: 3.55426, train_accuracy: 0.00
[1,   171] loss: 4.13177, train_accuracy: 0.00
[1,   181] loss: 2.73530, train_accuracy: 0.00
[1,   191] loss: 3.20715, train_accuracy: 25.00
[1,   201] loss: 2.31165, train_accuracy: 0.00


[1,  1731] loss: 2.54098, train_accuracy: 25.00
[1,  1741] loss: 2.44580, train_accuracy: 0.00
[1,  1751] loss: 1.66224, train_accuracy: 50.00
[1,  1761] loss: 2.57576, train_accuracy: 0.00
[1,  1771] loss: 2.43079, train_accuracy: 25.00
[1,  1781] loss: 2.01007, train_accuracy: 50.00
[1,  1791] loss: 2.90962, train_accuracy: 0.00
[1,  1801] loss: 2.44271, train_accuracy: 0.00
[1,  1811] loss: 1.98937, train_accuracy: 25.00
[1,  1821] loss: 2.52596, train_accuracy: 0.00
[1,  1831] loss: 2.39386, train_accuracy: 25.00
[1,  1841] loss: 2.80871, train_accuracy: 0.00
[1,  1851] loss: 1.81144, train_accuracy: 25.00
[1,  1861] loss: 1.76412, train_accuracy: 50.00
[1,  1871] loss: 2.04619, train_accuracy: 25.00
[1,  1881] loss: 2.03544, train_accuracy: 50.00
[1,  1891] loss: 2.19703, train_accuracy: 0.00
[1,  1901] loss: 1.19845, train_accuracy: 75.00
[1,  1911] loss: 1.88800, train_accuracy: 25.00
[1,  1921] loss: 2.84360, train_accuracy: 0.00
[1,  1931] loss: 2.37020, train_accuracy: 0.00
[

[2,  1061] loss: 1.91623, train_accuracy: 50.00
[2,  1071] loss: 2.70406, train_accuracy: 0.00
[2,  1081] loss: 2.24242, train_accuracy: 25.00
[2,  1091] loss: 1.91012, train_accuracy: 50.00
[2,  1101] loss: 2.16532, train_accuracy: 25.00
[2,  1111] loss: 2.11285, train_accuracy: 0.00
[2,  1121] loss: 1.92495, train_accuracy: 25.00
[2,  1131] loss: 2.13520, train_accuracy: 25.00
[2,  1141] loss: 2.04000, train_accuracy: 25.00
[2,  1151] loss: 2.35257, train_accuracy: 25.00
[2,  1161] loss: 2.38391, train_accuracy: 25.00
[2,  1171] loss: 2.17807, train_accuracy: 25.00
[2,  1181] loss: 2.18248, train_accuracy: 50.00
[2,  1191] loss: 1.87423, train_accuracy: 25.00
[2,  1201] loss: 1.56311, train_accuracy: 25.00
[2,  1211] loss: 1.87362, train_accuracy: 25.00
[2,  1221] loss: 1.93858, train_accuracy: 25.00
[2,  1231] loss: 1.82313, train_accuracy: 50.00
[2,  1241] loss: 1.49416, train_accuracy: 50.00
[2,  1251] loss: 2.52058, train_accuracy: 25.00
[2,  1261] loss: 1.67767, train_accuracy: 

[3,   391] loss: 2.17783, train_accuracy: 25.00
[3,   401] loss: 1.79315, train_accuracy: 50.00
[3,   411] loss: 2.16403, train_accuracy: 50.00
[3,   421] loss: 1.71988, train_accuracy: 0.00
[3,   431] loss: 1.66856, train_accuracy: 50.00
[3,   441] loss: 2.55641, train_accuracy: 0.00
[3,   451] loss: 2.65008, train_accuracy: 0.00
[3,   461] loss: 1.87365, train_accuracy: 25.00
[3,   471] loss: 2.11637, train_accuracy: 25.00
[3,   481] loss: 1.73038, train_accuracy: 50.00
[3,   491] loss: 1.96851, train_accuracy: 50.00
[3,   501] loss: 2.33217, train_accuracy: 0.00
[3,   511] loss: 2.01887, train_accuracy: 25.00
[3,   521] loss: 1.98082, train_accuracy: 25.00
[3,   531] loss: 2.37121, train_accuracy: 25.00
[3,   541] loss: 1.97533, train_accuracy: 50.00
[3,   551] loss: 2.37359, train_accuracy: 25.00
[3,   561] loss: 2.49579, train_accuracy: 0.00
[3,   571] loss: 1.62357, train_accuracy: 50.00
[3,   581] loss: 1.92344, train_accuracy: 50.00
[3,   591] loss: 2.12815, train_accuracy: 0.0

[3,  2111] loss: 1.50578, train_accuracy: 50.00
[3,  2121] loss: 1.26499, train_accuracy: 75.00
[3,  2131] loss: 1.89153, train_accuracy: 50.00
[3,  2141] loss: 2.45731, train_accuracy: 50.00
[3,  2151] loss: 1.66221, train_accuracy: 25.00
[3,  2161] loss: 1.76801, train_accuracy: 75.00
[3,  2171] loss: 2.34832, train_accuracy: 25.00
[3,  2181] loss: 2.31950, train_accuracy: 0.00
[3,  2191] loss: 1.32651, train_accuracy: 50.00
[3,  2201] loss: 1.83860, train_accuracy: 25.00
[3,  2211] loss: 1.88450, train_accuracy: 25.00
[3,  2221] loss: 2.99585, train_accuracy: 0.00
[3,  2231] loss: 1.74416, train_accuracy: 25.00
[3,  2241] loss: 1.70675, train_accuracy: 25.00
[3,  2251] loss: 2.17605, train_accuracy: 50.00
[3,  2261] loss: 1.94428, train_accuracy: 25.00
[3,  2271] loss: 1.48613, train_accuracy: 50.00
[3,  2281] loss: 1.55180, train_accuracy: 25.00
[3,  2291] loss: 2.00865, train_accuracy: 50.00
[3,  2301] loss: 1.40871, train_accuracy: 50.00
[3,  2311] loss: 1.96187, train_accuracy: 

...

[22,  2161] loss: 0.99997, train_accuracy: 75.00
[22,  2171] loss: 1.06310, train_accuracy: 75.00
[22,  2181] loss: 0.51636, train_accuracy: 100.00
[22,  2191] loss: 0.48578, train_accuracy: 100.00
[22,  2201] loss: 1.24845, train_accuracy: 50.00
[22,  2211] loss: 0.33211, train_accuracy: 100.00
[22,  2221] loss: 0.42281, train_accuracy: 75.00
[22,  2231] loss: 1.34181, train_accuracy: 75.00
[22,  2241] loss: 0.62959, train_accuracy: 75.00
[22,  2251] loss: 1.07303, train_accuracy: 75.00
[22,  2261] loss: 0.15764, train_accuracy: 100.00
[22,  2271] loss: 0.90206, train_accuracy: 75.00
[22,  2281] loss: 1.26609, train_accuracy: 75.00
[22,  2291] loss: 1.47688, train_accuracy: 50.00
[22,  2301] loss: 0.80588, train_accuracy: 75.00
[22,  2311] loss: 1.10949, train_accuracy: 50.00
[22,  2321] loss: 0.57951, train_accuracy: 75.00
[22,  2331] loss: 0.30137, train_accuracy: 100.00
[22,  2341] loss: 2.84707, train_accuracy: 25.00
[22,  2351] loss: 0.37834, train_accuracy: 100.00
[22,  2361] lo

[23,  1441] loss: 1.04006, train_accuracy: 75.00
[23,  1451] loss: 2.66952, train_accuracy: 25.00
[23,  1461] loss: 1.06781, train_accuracy: 75.00
[23,  1471] loss: 0.73605, train_accuracy: 75.00
[23,  1481] loss: 2.16592, train_accuracy: 50.00
[23,  1491] loss: 1.68553, train_accuracy: 75.00
[23,  1501] loss: 0.53983, train_accuracy: 100.00
[23,  1511] loss: 1.25893, train_accuracy: 50.00
[23,  1521] loss: 0.65636, train_accuracy: 75.00
[23,  1531] loss: 1.10949, train_accuracy: 50.00
[23,  1541] loss: 1.45246, train_accuracy: 50.00
[23,  1551] loss: 0.34161, train_accuracy: 75.00
[23,  1561] loss: 0.77614, train_accuracy: 75.00
[23,  1571] loss: 2.07434, train_accuracy: 50.00
[23,  1581] loss: 1.51816, train_accuracy: 25.00
[23,  1591] loss: 2.02334, train_accuracy: 25.00
[23,  1601] loss: 0.24580, train_accuracy: 100.00
[23,  1611] loss: 0.31738, train_accuracy: 75.00
[23,  1621] loss: 1.22959, train_accuracy: 75.00
[23,  1631] loss: 1.18483, train_accuracy: 75.00
[23,  1641] loss: 

[24,   721] loss: 1.44743, train_accuracy: 75.00
[24,   731] loss: 1.15660, train_accuracy: 50.00
[24,   741] loss: 1.66306, train_accuracy: 25.00
[24,   751] loss: 0.09651, train_accuracy: 100.00
[24,   761] loss: 0.96085, train_accuracy: 50.00
[24,   771] loss: 0.66886, train_accuracy: 100.00
[24,   781] loss: 2.38747, train_accuracy: 25.00
[24,   791] loss: 0.39366, train_accuracy: 100.00
[24,   801] loss: 1.44020, train_accuracy: 50.00
[24,   811] loss: 3.21952, train_accuracy: 25.00
[24,   821] loss: 2.53178, train_accuracy: 25.00
[24,   831] loss: 0.86566, train_accuracy: 50.00
[24,   841] loss: 1.58528, train_accuracy: 50.00
[24,   851] loss: 1.63973, train_accuracy: 50.00
[24,   861] loss: 0.65620, train_accuracy: 100.00
[24,   871] loss: 2.35964, train_accuracy: 0.00
[24,   881] loss: 0.65073, train_accuracy: 75.00
[24,   891] loss: 1.00166, train_accuracy: 25.00
[24,   901] loss: 1.85366, train_accuracy: 50.00
[24,   911] loss: 1.19213, train_accuracy: 50.00
[24,   921] loss:

[25,    11] loss: 0.88183, train_accuracy: 75.00
[25,    21] loss: 1.58380, train_accuracy: 50.00
[25,    31] loss: 0.56064, train_accuracy: 100.00
[25,    41] loss: 1.21349, train_accuracy: 75.00
[25,    51] loss: 1.60905, train_accuracy: 25.00
[25,    61] loss: 2.16892, train_accuracy: 50.00
[25,    71] loss: 2.01386, train_accuracy: 50.00
[25,    81] loss: 1.20961, train_accuracy: 50.00
[25,    91] loss: 1.56123, train_accuracy: 50.00
[25,   101] loss: 0.94206, train_accuracy: 75.00
[25,   111] loss: 1.24560, train_accuracy: 50.00
[25,   121] loss: 0.71203, train_accuracy: 75.00
[25,   131] loss: 1.32113, train_accuracy: 50.00
[25,   141] loss: 0.66068, train_accuracy: 100.00
[25,   151] loss: 0.48120, train_accuracy: 100.00
[25,   161] loss: 2.06703, train_accuracy: 0.00
[25,   171] loss: 0.60074, train_accuracy: 75.00
[25,   181] loss: 0.95654, train_accuracy: 75.00
[25,   191] loss: 0.82354, train_accuracy: 75.00
[25,   201] loss: 0.54398, train_accuracy: 75.00
[25,   211] loss: 

[25,  1681] loss: 3.82656, train_accuracy: 25.00
[25,  1691] loss: 1.19777, train_accuracy: 50.00
[25,  1701] loss: 0.45073, train_accuracy: 75.00
[25,  1711] loss: 2.64261, train_accuracy: 25.00
[25,  1721] loss: 0.96326, train_accuracy: 50.00
[25,  1731] loss: 2.15805, train_accuracy: 25.00
[25,  1741] loss: 0.31408, train_accuracy: 100.00
[25,  1751] loss: 0.91296, train_accuracy: 50.00
[25,  1761] loss: 1.31156, train_accuracy: 50.00
[25,  1771] loss: 0.86730, train_accuracy: 75.00
[25,  1781] loss: 1.32144, train_accuracy: 75.00
[25,  1791] loss: 0.36013, train_accuracy: 100.00
[25,  1801] loss: 1.29158, train_accuracy: 75.00
[25,  1811] loss: 1.29481, train_accuracy: 25.00
[25,  1821] loss: 0.90951, train_accuracy: 75.00
[25,  1831] loss: 2.91313, train_accuracy: 25.00
[25,  1841] loss: 1.21117, train_accuracy: 75.00
[25,  1851] loss: 1.09317, train_accuracy: 50.00
[25,  1861] loss: 0.58587, train_accuracy: 100.00
[25,  1871] loss: 0.46853, train_accuracy: 100.00
[25,  1881] loss